In [ ]:
const BFX = require('bitfinex-api-node');
var request = require('request');
const asTable   = require ('as-table');
const log   = require ('ololog').noLocate;
const ansi  = require ('ansicolor').nice;
const ccxt  = require ('ccxt');
const interval = require('interval-promise');
var { Series, DataFrame } = require('pandas-js');
const sqlite3 = require('sqlite3').verbose();
var util = require('util');
var nj = require('numjs');
const { google } = require('googleapis');
var CircularBuffer = require("circular-buffer");
const { Order } = require('bitfinex-api-node/lib/models');
const math = require('mathjs');

In [ ]:
let INTERVAL = 60;
let TABLE = "Bitfinex_900_15";
let QUOTE_ASSET = "BTC";
let ASSET_NUM = 15;
let HISTORIC_SIZE = 90;
let FEATURE_NUM = 3;
let VOLUME_AVERAGE_DAYS = 30;
let PROJECT='axiom-209217';
let MODEL='Bitfinex_900_15';
let VERSION='bitfinex_btc_900_6';

let EXCHANGE = "bitfinex";
let ORDER_TYPE= 'exchange market';
let WALLET = "exchange";
let KEY = 'KHYMo1wzIoU3uDcYn5qcO8lr3wHPVz5r3mcSFjr43XT';
let SECRET = "xQ0WERcmGjdLWAwPM4QV4jYyUbNzvwAleiawLQKlZJx";

let whitelisted_assets = [
    "XRP",
    "BCH",
    "ETH",
    "EOS",
    "LTC",
    "ETC",
    "XMR",
    "ZEC",
    "IOT",
    "NEO",
    "DSH",
    "ETP",
    "BTG",
    "ZRX",
    "TRX"
]
console.log(whitelisted_assets.length);

In [ ]:
const bfx = new BFX({
  apiKey: KEY,
  apiSecret: SECRET
})

In [ ]:
const restTwo = bfx.rest(2)

In [ ]:
const restOne = bfx.rest(1)

In [ ]:


// const get_top_assets = async function(){    
//     var symbols = await new Promise((resolve, reject) => {
//         restOne.get_symbols((err, res) => {
//             if (err) {
//                 console.log(err)
//                 reject(err)
//             }              
//             return resolve(res);
//         })
//     });
    
//     symbols.map(async (symbol) => {
//         if (symbol.endsWith(QUOTE_ASSET.toLowerCase())){             
//             console.log(symbol);
//         }
//     })
    
// }
// get_top_assets(15);

In [ ]:
// Return the amount of base currency that can be bought with one 
// quote currency
const get_ohlc = function(symbol) {
  return new Promise((resolve, reject) => {      
      request.get(
          ' https://api.bitfinex.com/v2/candles/trade:15m:'+symbol+'/hist?limit='+HISTORIC_SIZE,
          (err, response, body) => {
                   if (err) {
                        return reject(err)
                    }
                   return resolve(JSON.parse(body));
          }
       )      
  });
};

In [ ]:
const get_col = async function(arr,index) {
    let col = [];
    for (var r=0; r<arr.length; r++) {
           col[r] = arr[r][index];
    }
    return col;
}

In [ ]:
const construct_frame = async function(ohlc_data, asset_index) {

    
        return frame;
}

In [ ]:
(async function(){
      let ohlc = await get_ohlc( 'tXRP'+QUOTE_ASSET);
    

})();

In [ ]:
const construct_feature_frame = async function(assets) {    
    var close = [];
    var high = [];
    var low = [];
    var frame = [];
    
    assets.map(async (asset, asset_index) => {
        let ohlc = await get_ohlc( 't'+asset + QUOTE_ASSET);
        close[asset_index] = await get_col(ohlc, 2);
        high[asset_index] = await get_col(ohlc, 3);
        low[asset_index] =  await get_col(ohlc, 4);          
    });
    
    frame = [close,high,low]; 
}

## Reset Balances To Quote Asset

In [ ]:
// Sends 2 requests per invocation
const reset_position = async function() {

    const balances = await restTwo.balances() 

    // NORMALIZE BALANCES
    var orders = await Promise.all(
        await balances.map(async (balance) => {
        if (balance.type = WALLET){
                if (balance.currency !=QUOTE_ASSET.toLowerCase()){                    
                    return {
                          symbol: (balance.currency+QUOTE_ASSET).toUpperCase(),
                          amount: balance.available,
                          exchange: EXCHANGE,
                          price: '1000',
                          side: 'sell',
                          type: 'exchange market'
                    }
                }
        }        
    }));
    
    console.log(orders);
    
    restOne.multiple_new_orders(orders, (err, res) => {
      if (err) console.log(err)
      console.log(res)
    })
        
}
reset_position();

## Order Execution Logic

In [ ]:
// Return the amount of base currency that can be bought with one 
// quote currency
const get_last_price = function(symbol) {
  return new Promise((resolve, reject) => {
    restOne.ticker(symbol, (err, res) => {
        if (err) {
            return reject(err)
        }
        return resolve(1/parseFloat(res.last_price))
    }) 
  })
}


In [ ]:
// Sends 3 requests per invocation
const execute_position = async function(pv) {
    
    let total_balance=0;
    
    const balances = await restTwo.balances()
   
    balances.map(async (balance) => {
                if (balance.type = WALLET){
                    if (balance.currency ==QUOTE_ASSET.toLowerCase()){         
                           total_balance += balance.available;
                    }
                }
    });
    
    console.log(total_balance);
    
    // TODO check that all orders are fulfilled and all balance is in lossless asset
    // TODO assert that balance is not 0
    var orders =  await Promise.all(
        await pv.map(async (ps) => {
             return {
                  symbol: ps.symbol,
                  amount: ps.fraction * (total_balance * await get_last_price(ps.symbol)),
                  exchange: EXCHANGE,
                  price: '1000',
                  side: 'buy',
                  type: 'exchange market'
            }
    }));
    
    console.log(orders);

    
    restOne.multiple_new_orders(orders, (err, res) => {
      if (err) console.log(err)
      console.log(res)
    })
        
}

const pv = [
        {symbol:'XRPBTC', fraction:0.4},
];

execute_position(pv);

# PREDICT AND PLACE ORDERS

Periodically request latest [high, low, close] set from exchange ticker or  OHLC chart API and send historic buffer and previous portfolio weight vector to google cloud mahcine learning engine model, once the prediction has been received from the machine learning model create the neccessary orders to fill the position therein

In [ ]:
//   [
//     SYMBOL,
//     BID, 
//     BID_SIZE, 
//     ASK, 
//     ASK_SIZE, 
//     DAILY_CHANGE, 
//     DAILY_CHANGE_PERC, 
//     LAST_PRICE, 
//     VOLUME, 
//     HIGH, 
//     LOW
//   ],



(async function() {
    let reset_iteration = false
    let top_assets = {}
    
    // GET FEATURE FRAME
    
     // GET PREDICTION FROM GOOGLE CLOUD MACHINE LEARNING ENGINE 
    google.auth.getApplicationDefault( (err, authClient, projectId) => {
        if (err) {
            console.log('Authentication failed because of ', err);
            res.status(401).send('Authentication failed');

        } else {

            var ml = google.ml({
                version: 'v1',
                auth: authClient
            });

           var instances ={'instances': [{
               'input': inp,
               'previous_w':prev_w,
           }]};

            ml.projects.predict({
                name: util.format('projects/%s/models/%s/versions/%s', PROJECT, MODEL, VERSION),
                resource: instances
            }, 
            function(err, result) {
                if (err) {
                    return callback(err);
                }

                // FLATTEN: CLOSE ALL OPEN ORDERS AND POSITIONS

                var payload = {
                   request: '/v1/order/cancel/all',
                   nonce: Date.now().toString(),
                }
           });
       }
    });
    
    // CANCEL ALL ORDERS
    bfxRest.cancel_all_orders((err, res) => {
      if (err) console.log(err)
      console.log(result)
    })

    // RESET BALANCES TO QUOTE ASSET FOR REDISTRIBUTION
    
    
    // 
    
})();